In [1]:
import pandas as pd
import numpy as np

In [2]:
#read data
calories2 = pd.read_json('../osfstorage-archive/pmdata/p02/fitbit/calories.json')
distance2 = pd.read_json('../osfstorage-archive/pmdata/p02/fitbit/distance.json')
l_act2 = pd.read_json('../osfstorage-archive/pmdata/p02/fitbit/lightly_active_minutes.json')
m_act2 = pd.read_json('../osfstorage-archive/pmdata/p02/fitbit/moderately_active_minutes.json')
r_heart2 = pd.read_json('../osfstorage-archive/pmdata/p02/fitbit/resting_heart_rate.json')
sedentary2 = pd.read_json('../osfstorage-archive/pmdata/p02/fitbit/sedentary_minutes.json')
sleep_score2 = pd.read_csv('../osfstorage-archive/pmdata/p02/fitbit/sleep_score.csv')
sleep2 = pd.read_json('../osfstorage-archive/pmdata/p02/fitbit/sleep.json')
steps2 = pd.read_json('../osfstorage-archive/pmdata/p02/fitbit/steps.json')
hr_zone2 = pd.read_json('../osfstorage-archive/pmdata/p02/fitbit/time_in_heart_rate_zones.json')
v_act2 = pd.read_json('../osfstorage-archive/pmdata/p02/fitbit/very_active_minutes.json')

In [3]:
#data frequency to date using sum
calories_processed = calories2.resample('D', on = 'dateTime').value.sum()
distance_processed = distance2.resample('D', on = 'dateTime').value.sum()
steps_processed = steps2.resample('D', on = 'dateTime').value.sum()

In [4]:
#resting heart rate to simple df
r_heart2['value']=r_heart2['value'].apply(lambda x: x['value'])
r_heart_processed=r_heart2

In [5]:
#delete time in sleep score time stamp
sleep_score2['timestamp']=sleep_score2['timestamp'].apply(lambda x: x[:10])
#drop unnecessary column
sleep_score_processed = sleep_score2.drop(columns='sleep_log_entry_id') #drop entry id

In [6]:
#dictionary with heart rate zone minutes to df
hr_zone_value = pd.DataFrame([eval(str(a).replace('{\'valuesInZones\': ','').replace('}}','}')) for a in hr_zone2['value']])
hr_zone_processed = pd.concat([hr_zone2['dateTime'], hr_zone_value], axis=1)

In [7]:
#unify index
l_act_processed = l_act2.set_index('dateTime')
m_act_processed = m_act2.set_index('dateTime')
r_heart_processed = r_heart_processed.set_index('dateTime')
sedentary_processed = sedentary2.set_index('dateTime')
sleep_score_processed = sleep_score_processed.set_index('timestamp')
sleep_processed = sleep2.set_index('dateOfSleep')
hr_zone_processed = hr_zone_processed.set_index('dateTime')
v_act_processed = v_act2.set_index('dateTime')
#index to DateTimeIndex
sleep_score_processed.index = pd.to_datetime(sleep_score_processed.index)

In [8]:
#get dummy values for mainsleep true(1)/false(0)
sleep_processed['mainSleep'] = pd.get_dummies(sleep_processed['mainSleep'])[True].fillna(0)
#drop unnecessary columns
sleep_processed = sleep_processed.drop(['logId','startTime','endTime','type','infoCode','levels'], axis=1)
#index to DateTimeIndex
sleep_processed.index = pd.to_datetime(sleep_processed.index)

In [9]:
#read prediction target
wellness = pd.read_csv('../osfstorage-archive/pmdata/p02/pmsys/wellness.csv') 
wellness['effective_time_frame'] = wellness['effective_time_frame'].apply(lambda x: x[:10])
wellness = wellness.rename(columns={"effective_time_frame": "dateTime"})
readiness = wellness.loc[:,['dateTime','readiness']].set_index('dateTime')
readiness.index = pd.to_datetime(readiness.index)

In [10]:
#drop duplicated records
readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed = [x[~x.index.duplicated(keep='first')] for x in [readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed]]

In [11]:
#concat data
P02 = pd.concat([readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed], axis=1)

In [12]:
#rename columns
P02.columns=['readiness','calories','distance','l_act','m_act','resting_heart','sedentary','sleep_overall_score','sleep_composition_score',
             'sleep_revitalization_score', 'sleep_duration_score',
             'deep_sleep_in_minutes', 'sleep_resting_heart_rate','sleep_restlessness',
             'sleep_duration','sleep_minutesToFallAsleep','sleep_minutesAsleep','sleep_minutesAwake',
             'sleep_minutesAfterWakeup','sleep_timeInBed','sleep_efficiency',
             'main_sleep','steps',
             'hr_below_zone1', 'hr_zone1', 'hr_zone3','hr_zone2','v_act']

In [13]:
P02

,readiness,calories,distance,l_act,m_act,resting_heart,sedentary,sleep_overall_score,sleep_composition_score,sleep_revitalization_score,...,sleep_minutesAfterWakeup,sleep_timeInBed,sleep_efficiency,main_sleep,steps,hr_below_zone1,hr_zone1,hr_zone3,hr_zone2,v_act
2019-11-01,NaN,1670.40,NaN,0,0,NaN,1440,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2019-11-02,NaN,1670.40,NaN,0,0,NaN,1440,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2019-11-03,8.0,1670.40,NaN,0,0,NaN,1440,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2019-11-04,NaN,1670.40,NaN,0,0,NaN,1440,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2019-11-05,NaN,1778.26,27110.0,16,0,NaN,1424,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,385.0,9.0,66.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-27,NaN,3290.23,1120640.0,171,12,47.367889,680,65.0,20.0,16.0,...,0.0,486.0,98.0,1.0,15078.0,59.0,1264.0,35.0,6.0,91
2020-03-28,NaN,3533.11,1277910.0,237,42,49.407214,609,62.0,19.0,11.0,...,0.0,443.0,89.0,1.0,17133.0,147.0,1279.0,1.0,0.0,109
2020-03-29,3.0,3028.75,972940.0,77,11,51.050967,983,59.0,18.0,14.0,...,0.0,254.0,92.0,1.0,13039.0,140.0,799.0,2.0,0.0,115
2020-03-30,3.0,3407.56,1370520.0,96,20,48.865365,784,77.0,21.0,21.0,...,0.0,388.0,93.0,1.0,18485.0,137.0,1253.0,19.0,0.0,152


In [14]:
P02['main_sleep'].value_counts()

1.0    138
0.0      4
Name: main_sleep, dtype: int64